In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

import os 
import shutil

import contactProbability_generator_with_stiffness_new as cmgsp
from contactProbability_generator_with_stiffness_new import ChainLayout

from mirnylib.numutils import zoomArray
import sys
sys.setrecursionlimit(4000)

from multiprocessing import Pool
import pickle

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from mirnylib.numutils import iterativeCorrection
from itertools import product

from scipy.ndimage.filters import gaussian_filter
import matplotlib.colors as clr 

import timeit
from scipy.sparse import coo_matrix
from mirnylib.numutils import logbins
from scipy.ndimage.filters import gaussian_filter1d

## Distributions that will be used
from numpy.random import exponential 

## simulation parameters
chainLen = 10000 # 1 kb monomers
monToBP = 2000 

def generate_chain(L_tot,loopargs,gapargs):
    chain  = []
    c_count = 0
    
    while c_count < L_tot:
        # generate loop distributions
        keys = loopargs.keys()
        if 'exponential' in keys:
            lavg = loopargs['exponential']['avg']
            lsize = int(exponential(lavg)) # cast to integer
            if lsize < 1:
                lsize = 1
            if c_count + lsize < L_tot:
                chain.append((c_count,c_count+lsize))
            c_count += lsize

        else:
            raise ValueError("loopargs does not have proper arguments")

        # generate gap distribution
        keys = gapargs.keys()
        if 'exponential' in keys:
            gavg = gapargs['exponential']['avg']
            gsize = int(exponential(gavg)) # cast to integer
            if gsize < 1: 
                gsize = 1
            c_count += gsize
        else:
            raise ValueError("loopargs does not have proper arguments")
    
    return chain


## fast sampling 

def doOneChain(L_tot,loopargs,gapargs,nsamples=60,nreps=1000,H=1/2,lp=1,circular=False):

    X = []
    Y = []
    P = []
    C = []
    
    for ic in range(nreps):
        # generate chain using loop and gap statistics
        smcs = generate_chain(L_tot,loopargs,gapargs)
        cl = None
        try:
            if len(smcs)==0:
                smcs = [(0,1)]
            cl = ChainLayout(smcs,L_tot)
        except:
            print("Pseudoknot formed count:{}".format(ic))
            print(cl)
            assert(1==0)
            cl = None
        if cl is None:
            continue

        # subsample from distribution 
        vals = sorted(np.random.choice(L_tot,nsamples, replace=False))
        for ix in range(len(vals)):
            for iy in range(ix,len(vals)):
                x = vals[ix]
                y = vals[iy]
                deff = cl.get_dist(x,y,H=H,lp=lp,circular=circular)
                if deff == 0:
                    pc = 1
                else:
                    pc = 1/np.sqrt(deff)**3
                if not np.isnan(pc):
                    X.append(x)
                    Y.append(y) # x
                    P.append(pc) # probability
                    C.append(1) # counts

    Z = coo_matrix((P,(X,Y)),shape=(L_tot,L_tot))
    Zcount = coo_matrix((C,(X,Y)),shape=(L_tot,L_tot))

    return Z,Zcount, (P,X,Y)


def get_deriv(res, lbins, filter_width):
    
    
    lbins = np.array(lbins)
    
    ## combine coo matrices
    dS = np.concatenate([Z[0].col-Z[0].row for Z in res])
    Zdata = np.concatenate([Z[0].data for Z in res])
    Zcountdata = np.concatenate([Z[1].data for Z in res])

    # do log binning of data
    rebinned_dS = np.digitize(dS,lbins,right=True) 
    freq = np.bincount(rebinned_dS,Zdata)
    count_freq = np.bincount(rebinned_dS,Zcountdata)
    Pc_s = freq/count_freq
    Pc_s = Pc_s[1:]
    
    bins = [(lbins[i], lbins[i + 1]) for i in range(len(lbins) - 1)]
    mids = [np.sqrt(i[0] * (i[1] - 1)) for i in bins]
    
    dlogP = np.diff(np.log(Pc_s))
    dlogS = np.diff(np.log(mids))
    
    H = dlogP/dlogS
    return np.array(mids),gaussian_filter1d(H,filter_width)

def get_pc(res, lbins, filter_width):
    
    lbins = np.array(lbins)

    ## combine coo matrices
    dS = np.concatenate([Z[0].col-Z[0].row for Z in res])
    Zdata = np.concatenate([Z[0].data for Z in res])
    Zcountdata = np.concatenate([Z[1].data for Z in res])

    # do log binning of data
    rebinned_dS = np.digitize(dS,lbins,right=True) 
    freq = np.bincount(rebinned_dS,Zdata)
    count_freq = np.bincount(rebinned_dS,Zcountdata)
    Pc_s = freq/count_freq
    Pc_s = Pc_s[1:]
    
    bins = [(lbins[i], lbins[i + 1]) for i in range(len(lbins) - 1)]
    mids = [np.sqrt(i[0] * (i[1] - 1)) for i in bins]

    return np.array(mids),Pc_s

In [3]:
# chain length
chainLen = 10000

# binning
from mirnylib.numutils import logbinsnew
bins = logbinsnew(1, chainLen, 1.3)

# parameters of the model
H = 1/3
lavg = 100
gavg = 50
lp = 2

# loops and gaps distribution
loopargs= {'exponential': {'avg': lavg }}
gapargs = {'exponential': {'avg': gavg }}

# sampling statistics
nsamples=100
nreps=5000
num_cores=30

In [ ]:
# running
with Pool(num_cores) as mypool:
    res = mypool.starmap(doOneChain, list(product([chainLen], [loopargs]*num_cores, [gapargs],[nsamples],[nreps],[H],[lp],[False]  )))
mypool.close()

# compute P(s) and slopes from generated dataframes
mids, pc = get_pc(res, bins, filter_width=1)
mids, Hder = get_deriv(res, bins, filter_width=1)

# free up space
del res

In [ ]:
from matplotlib import gridspec
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
font = {'family' : 'Arial',
    'weight' : 'medium',
    'size'   : 14,
    'style'  : 'normal'}
matplotlib.rc('font', **font)


color_idx = np.linspace(0, 1, 6)

fig = plt.figure(figsize=(5, 7)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

ax0.plot(mids, pc, color='k', linewidth=2)

ax0.set_xscale('log')
ax0.set_yscale('log')

ax1.plot(mids[1:], Hder, color='k', linewidth=2)

ax1.set_xscale('log')

plt.show()